In [9]:
import os
import time
import psutil
import pandas as pd
import mysql.connector
import re
from datetime import datetime, timedelta
from mysql.connector import Error
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from mysql.connector import Error


load_dotenv()
HOST = os.getenv('MYSQL_HOST')
USER = os.getenv('MYSQL_USER')
PASSWORD = os.getenv('MYSQL_PASSWORD')
DATABASE = os.getenv('MYSQL_DATABASE')
PORT = int(os.getenv('MYSQL_PORT'))
JADWAL_SWITCH_PARAM = {
                'prototipe':{'normal':timedelta(minutes=0.3),'eskalasi':timedelta(minutes=0.1)},
                'production':{'normal':timedelta(days=7),'eskalasi':timedelta(days=3)}
                 }
JADWAL_SWITCH_OPERATOR = JADWAL_SWITCH_PARAM.get('prototipe')
# jadwal_selanjutnya = time_now + timedelta(minutes=5)  

class WhatsAPI:
    def __init__(self):
        self.driver_path = r'edge web driver\msedgedriver.exe'
        self.binary_path = r'C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe'
        self.user_data_dir = r'C:\Users\eka agung\AppData\Local\Microsoft\Edge\User Data'
        self.profile_directory = 'Default'

    def check_edge_process(self):
        return any(proc.info['name'] == 'msedge.exe' for proc in psutil.process_iter(['name']))

    def terminate_edge_process(self):
        for proc in psutil.process_iter(['name']):
            if proc.info['name'] == 'msedge.exe':
                try:
                    proc.terminate()
                    proc.wait(timeout=5)
                    print(f"Terminated Edge process with PID {proc.pid}")
                except Exception as e:
                    print(f"Failed to terminate process {proc.pid}: {e}")

    def get_driver(self):
        options = Options()
        options.binary_location = self.binary_path
        options.add_argument(f"user-data-dir={self.user_data_dir}")
        options.add_argument(f"profile-directory={self.profile_directory}")
        options.add_argument("--disable-infobars")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--start-maximized")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        service = Service(executable_path=self.driver_path)
        return webdriver.Edge(service=service, options=options)

    def check_login_QR(self, driver):
        return len(driver.find_elements(By.XPATH, "//canvas[@aria-label='Scan this QR code to link a device!' and @role='img']"))

    def check_app_initialize_screen(self, driver):
        return len(driver.find_elements(By.XPATH, "//div[@id='wa_web_initial_startup' and @class='_apdl']"))

    def check_chat_icon(self, driver):
        return len(driver.find_elements(By.XPATH, "//div[@class='x1c4vz4f xs83m0k xdl72j9 x1g77sc7 x78zum5 xozqiw3 x1oa3qoh x12fk4p8 xeuugli x2lwn1j x1nhvcw1 x1q0g3np x6s0dn4 xh8yej3']"))

    def get_profile_name_elements(self, driver):
        return driver.find_elements(By.XPATH, "//span[@dir='auto' and @class='x1iyjqo2 x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft x1rg5ohu _ao3e']")

    def process_profile_name_element(self, element):
        html = element.get_attribute("outerHTML")
        soup = BeautifulSoup(html, 'html.parser')
        text = soup.get_text(strip=True)
        return text

    def wait_for_dom_stable(self, driver, timeout=30, check_interval=0.5):
        driver.execute_script("""
            window.domChanged = false;
            const observer = new MutationObserver(() => {
                window.domChanged = true;
            });
            observer.observe(document.body, {
                childList: true,
                subtree: true,
                attributes: true
            });
        """)
        end_time = time.time() + timeout
        while time.time() < end_time:
            changed = driver.execute_script("return window.domChanged;")
            if not changed:
                return True
            driver.execute_script("window.domChanged = false;")
            time.sleep(check_interval)
        return False
    def check_button(self,driver,xpath:str):
        return driver.find_elements(By.XPATH, xpath)
        
    def tunggu_dan_klik_button(self, driver, class_name="x", timeout=30):
        try:
            tombol = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CLASS_NAME, class_name))
            )
            tombol.click()
            print(f"✅ Tombol dengan class '{class_name}' berhasil diklik.")
            return True
        except TimeoutException:
                return False
        except Exception as e:
            print(f"❌ Gagal klik tombol dengan class '{class_name}': {e}")

    def scroll_message(self, driver, times=1):
        try:
            scrollable_div = driver.find_element(
                By.XPATH,
                '//div[@class="x10l6tqk x13vifvy x1o0tod xyw6214 x9f619 x78zum5 xdt5ytf xh8yej3 x5yr21d x6ikm8r x1rife3k xjbqb8w x1ewm37j" and @tabindex="0"]'
            )
            for _ in range(times):
                driver.execute_script("arguments[0].scrollTop = 0;", scrollable_div)
        except NoSuchElementException:
            print("Elemen tidak ditemukan. Melakukan refresh halaman...")
            driver.refresh()
            time.sleep(3)

    def get_text_data(self,driver):
        data = {}
        incoming_messages = driver.find_elements(By.XPATH, '//div[@tabindex="-1" and @role="row"]/div[@data-id]')

        for msg in incoming_messages:
            try:
                data_id = msg.get_attribute('data-id')
                msg_in = msg.find_element(By.XPATH,'.//div[contains(@class, "message-in")]')
                pre_plain_elem = msg_in.find_element(By.XPATH,'.//div[contains(@class, "copyable-text") and @data-pre-plain-text]')
                pre_plain_text = pre_plain_elem.get_attribute('data-pre-plain-text')
                match = re.search(r'\[(.*?)\]', pre_plain_text)
                if not match:
                    continue
                timestamp = datetime.strptime(match.group(1), "%H.%M, %d/%m/%Y")
                text_content = pre_plain_elem.text
                
            except Exception as e:
                continue
            if re.search(r'https?://\S+|www\.\S+', text_content) or len(text_content)==0:
                continue
            # continue
            if timestamp not in data:
                data[timestamp] = {}
            if data_id not in data[timestamp]:
                data[timestamp][data_id] = []
            if text_content and text_content not in data[timestamp][data_id]:
                data[timestamp][data_id].append(text_content)
        #del incoming_messages,data_id,pre_plain_elem,pre_plain_text,timestamp,match,msg_in,text_content
        return data
    def check_new_respon(self,driver,waktu_terakhir_kirim_permintaan):
        start_time = time.time()
        while True:
            # Ambil data terbaru dari driver
            data_text = self.get_text_data(driver)
            
            
            # Cek apakah ada data BARU (lebih baru dari waktu referensi)
            found_new = False
            for timestamp in data_text.keys():
                if timestamp < waktu_terakhir_kirim_permintaan:
                    found_new = True
                    break  # Keluar dari loop for begitu ditemukan satu data baru
            
            # Jika ditemukan data baru, keluar dari loop while
            if found_new:
                break
            
            # Jika tidak ada data baru, tunggu sebentar sebelum cek ulang
            
          
            if time.time() - start_time > 15:
                return None
            self.scroll_message(driver)
        return data_text
def make_cursor():
        try:
            connection = mysql.connector.connect(
                host=HOST or os.getenv("DB_HOST"),
                user=USER or os.getenv("DB_USER"),
                password=PASSWORD or os.getenv("DB_PASSWORD"),
                database=DATABASE or os.getenv("DB_NAME"),
                port=PORT or int(os.getenv("DB_PORT", 3306))
            )
            cursor = connection.cursor(dictionary=True)
            return cursor, connection
        except Error as e:
            raise ConnectionError(f"❌ Gagal membuat koneksi database: {e}")

def load_data_koresponden(connection):
    table = 'data_koresponden'
    try:
        if connection.is_connected():
            query = f"SELECT * FROM {table};"
            df = pd.read_sql(query, connection)
            return df
    except Error as e:
        print("❌ Gagal koneksi:", e)
        return pd.DataFrame()
    finally:
        if connection.is_connected():
            connection.close()
            print("🔌 Koneksi ditutup")

def format_wa_number(no_wa: str) -> str:
    """
    Jika nomor diawali dengan '0', ganti menjadi '61' (kode negara Australia).
    """
    no_wa = no_wa.strip()
    if no_wa.startswith("0"):
        return "62" + no_wa[1:]
    return no_wa


def update_or_insert_schedule(cursor, no_wa, next_date):
    query_check = "SELECT * FROM jadwal_pengiriman_pesan_selanjutnya WHERE no_wa = %s"
    cursor.execute(query_check, (no_wa,))
    result = cursor.fetchone()
    if result:
        query_update = """
            UPDATE jadwal_pengiriman_pesan_selanjutnya 
            SET jadwal_pengiriman_pesan_selanjutnya = %s 
            WHERE no_wa = %s
        """
        cursor.execute(query_update, (next_date, no_wa))
    else:
        query_insert = """
            INSERT INTO jadwal_pengiriman_pesan_selanjutnya (no_wa, jadwal_pengiriman_pesan_selanjutnya) 
            VALUES (%s, %s)
        """
        cursor.execute(query_insert, (no_wa, next_date))

def kirim_pesan_permintaan(driver, pesan_kirim: str):
    try:
        input_box = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//div[@contenteditable="true" and @aria-label="Ketik pesan"]'
            ))
        )
        try:
            if not pesan_kirim:  # Cek jika variabel kosong atau None
                raise ValueError("❗ pesan_kirim kosong. Harap isi pesan terlebih dahulu.")
            pesan = pesan_kirim
        except NameError:
            raise NameError("❗ pesan_kirim belum didefinisikan.")
        input_box.click()
        input_box.send_keys(pesan)
        input_box.send_keys(Keys.ENTER)
        print("📩 Permintaan data terkirim.")
        return True
    except Exception as e:
        print(f"❌ Gagal mengirim pesan: {e}")
        return False

def update_text_data(data_text,no_wa):
    valid_data = False
    if len(data_text)>0:
        for date in data_text.keys():
            for data_id in data_text[date]:
                # Cek duplikasi di tabel respon_koresponden
                cursor.execute(
                    """
                    SELECT COUNT(*) as jumlah FROM respon_koresponden 
                    WHERE no_wa = %s AND pukul_respon = %s AND data_id = %s AND tipe_respon = %s
                    """,
                    (no_wa, date, data_id, 'text')
                )
                result = cursor.fetchone()
                if result['jumlah'] == 0:
                    cursor.execute(
                        "INSERT INTO respon_koresponden (no_wa, pukul_respon, data_id, tipe_respon) VALUES (%s, %s, %s, %s)",
                        (no_wa, date, data_id, 'text')
                    )
                    conn.commit()

                # Ambil semua text_content (list) untuk data_id ini
                list_text = data_text[date][data_id]
                
                for text_var in list_text:
                    
                    # Cek duplikasi di tabel text_respon
                    cursor.execute(
                        """
                        SELECT COUNT(*) as jumlah FROM text_respon 
                        WHERE data_id = %s AND text_respon = %s
                        """,
                        (data_id, text_var)
                    )
                    result = cursor.fetchone()
                    if result['jumlah'] == 0:
                        cursor.execute(
                            "INSERT INTO text_respon (data_id, text_respon) VALUES (%s, %s)",
                            (data_id, text_var)
                        )
                        conn.commit()
                        valid_data = True
        # cursor.execute("UPDATE log_pengiriman_permintaan SET respon = %s WHERE respon = %s AND no_wa = %s",(True,False,no_wa))
        # conn.commit()
    return valid_data


    

In [10]:
WA_API = WhatsAPI()
base_url_wa = 'https://' +"web.whatsapp.com"
if WA_API.check_edge_process():
    WA_API.terminate_edge_process()
else:
    pass
DRIVER = WA_API.get_driver()
DRIVER.get(base_url_wa)
while True:
    if WA_API.wait_for_dom_stable(DRIVER,timeout=int(os.getenv('SCAN_TIMEOUT'))):
        while True:
            if WA_API.check_login_QR(DRIVER) == 0 and WA_API.check_app_initialize_screen(DRIVER) == 0 and WA_API.check_chat_icon(DRIVER)>0:
                time.sleep(6)
                
                # WA_API.tunggu_dan_klik_button(DRIVER,class_name="x889kno x1a8lsjc x13jy36j x64bnmy x1n2onr6 x1rg5ohu xk50ysn x1f6kntn xyesn5m x1rl75mt x19t5iym xz7t8uv x13xmedi x178xt8z x1lun4ml xso031l xpilrb4 x13fuv20 x18b5jzi x1q0q8m5 x1t7ytsu x1v8p93f x1o3jo1z x16stqrj xv5lvn5 x1hl8ikr xfagghw x9dyr19 x9lcvmn x1pse0pq xcjl5na xfn3atn x1k3x3db x9qntcr xuxw1ft xv52azi")
                
                break
            else:
                time.sleep(1)
                continue
        break

    else:
        continue
    
# try:
#     tombol = DRIVER.find_element(
#         By.XPATH,
#         '//button[@class="x889kno x1a8lsjc x13jy36j x64bnmy x1n2onr6 x1rg5ohu xk50ysn x1f6kntn xyesn5m x1rl75mt x19t5iym xz7t8uv x13xmedi x178xt8z x1lun4ml xso031l xpilrb4 x13fuv20 x18b5jzi x1q0q8m5 x1t7ytsu x1v8p93f x1o3jo1z x16stqrj xv5lvn5 x1hl8ikr xfagghw x9dyr19 x9lcvmn x1pse0pq xcjl5na xfn3atn x1k3x3db x9qntcr xuxw1ft xv52azi"]'
#     )
#     tombol.click()
#     print("✅ Tombol diklik.")
# except NoSuchElementException:
#     print("❌ Tombol tidak ditemukan.")
#     pass


cursor, conn  = make_cursor()
df = load_data_koresponden(conn)
jenis_data = {
    'pns': 'data konsumsi ikan, harga ikan hari ini, blablabla',
    'nelayan': 'data hasil tangkapan laut, harga solar, blablabla',
    'petani': 'data luas tambak, panen udang, blablabla',
    # tambahkan lainnya sesuai kebutuhan
}
print(df.head())
# while True:
no_wa = df['no_wa'].iat[0]
pekerjaan = df['pekerjaan'].iat[0]
formatted_wa = format_wa_number(no_wa)
url = f"https://web.whatsapp.com/send?phone={formatted_wa}&source=&data=#"
DRIVER.get(url)
while True:
    if WA_API.wait_for_dom_stable(DRIVER,timeout=int(os.getenv('SCAN_TIMEOUT'))):
        break
    else:
        continue

print(f"📨 Membuka chat: {no_wa}")

Terminated Edge process with PID 1912
Terminated Edge process with PID 2488
Terminated Edge process with PID 4980
Terminated Edge process with PID 8940
Terminated Edge process with PID 17912
Terminated Edge process with PID 18412
Terminated Edge process with PID 20148
Terminated Edge process with PID 20180


C:\Users\eka agung\AppData\Local\Temp\ipykernel_23700\396791669.py:210: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


🔌 Koneksi ditutup
   id nama_korespoden         no_wa  kecamatan     desa pekerjaan
0   1       prototipe  085218274543  kota maba  sangaji       pns
📨 Membuka chat: 085218274543


In [11]:
try:
    cursor, conn = make_cursor()  # asumsi kamu sudah punya fungsi ini

    # Hapus semua data dari tabel
    cursor.execute("DELETE FROM cakalang.log_pengiriman_permintaan;")
    cursor.execute("DELETE FROM cakalang.jadwal_pengiriman_pesan_selanjutnya;")
    cursor.execute("DELETE FROM cakalang.respon_koresponden;")
    cursor.execute("DELETE FROM cakalang.text_respon;")

    # Reset AUTO_INCREMENT jika dibutuhkan (opsional)
    # cursor.execute("ALTER TABLE cakalang.log_pengiriman_permintaan AUTO_INCREMENT = 1;")
    # cursor.execute("ALTER TABLE cakalang.jadwal_pengiriman_pesan_selanjutnya AUTO_INCREMENT = 1;")
    # cursor.execute("ALTER TABLE cakalang.respon_koresponden AUTO_INCREMENT = 1;")
    # cursor.execute("ALTER TABLE cakalang.text_respon AUTO_INCREMENT = 1;")

    conn.commit()
    print("✅ Semua data berhasil dihapus dan AUTO_INCREMENT direset.")
except Exception as e:
    print(f"❌ Gagal menghapus data: {e}")
finally:
    cursor.close()
    conn.close()
# # cursor.execute(
# #     """
# #     SELECT COUNT(*) AS jumlah_belum_respon FROM log_pengiriman_permintaan
# #     WHERE no_wa = %s AND respon = %s
# #     """,
# #     (no_wa, 0)
# # )
# # result = cursor.fetchone()
# # result

✅ Semua data berhasil dihapus dan AUTO_INCREMENT direset.


In [13]:
try:
    while True:
        # proses utama kamu di sini
        print("Menjalankan proses... (Tekan Ctrl+C untuk berhenti)")
        # delay biar tidak spam
        time.sleep(6)

        jenis_data = {'pns':'konsumsi ikan berupa harga beli dan jumlah ikan yang dibeli'}


        cursor, conn  = make_cursor()
        cursor.execute(
            "SELECT MAX(jadwal_pengiriman_pesan_selanjutnya) AS jadwal FROM jadwal_pengiriman_pesan_selanjutnya WHERE no_wa = %s",
            (no_wa,)
        )

        result = cursor.fetchone()
        jadwal = result['jadwal'] if result and result['jadwal'] else None
        # # time_now = datetime.now().strftime("%d/%m/%Y")
        # cursor.execute("SELECT MAX(pukul_respon) AS terakhir FROM respon_koresponden WHERE no_wa = %s", (no_wa,))
        # result = cursor.fetchone()
        # waktu_terakhir_respon = result['terakhir'] if result and result['terakhir'] else None

        # cursor.execute("SELECT MAX(waktu_pengiriman) AS terakhir_kirim FROM log_pengiriman_permintaan WHERE no_wa = %s", (no_wa,))
        # result = cursor.fetchone()
        # waktu_terakhir_kirim_permintaan = result['terakhir_kirim'] if result and result['terakhir_kirim'] else None
        time_now = datetime.now()
        # print(str(time_now)+' | '+str(jadwal))
        if jadwal is None:
            print('jadwal '+str(jadwal))
            # tetap sebagai objek datetime
            jadwal_selanjutnya = time_now + JADWAL_SWITCH_OPERATOR.get('normal') 
            pesan = f'ini adalah pesan pertama untuk pengambilan data koresponden {no_wa} pertama kali, data yang diharapkan {jenis_data.get(pekerjaan)}'
            kirim_pesan_permintaan(DRIVER,pesan)
            # perbarui log permintaan
            try:
                cursor.execute(
                    "INSERT INTO log_pengiriman_permintaan (no_wa, waktu_pengiriman,respon) VALUES (%s, %s, %s)",
                    (no_wa, time_now, 0)
                )
                conn.commit()
            except Exception as e:
                print(f"❌ Gagal insert respon: {e}")
            # perbarui jadwal
            try:
                cursor.execute(
                    "INSERT INTO jadwal_pengiriman_pesan_selanjutnya (no_wa, jadwal_pengiriman_pesan_selanjutnya) VALUES (%s, %s)",
                    (no_wa, jadwal_selanjutnya)
                )
                
                conn.commit()
                print('penyimpanan log permintaan berhasil')
            except Exception as e:
                print(f"❌ Gagal insert Jadwal: {e}")

        elif jadwal is not None and time_now >= jadwal:
            
            cursor.execute(
                """
                SELECT MIN(waktu_pengiriman) FROM log_pengiriman_permintaan 
                WHERE no_wa = %s AND respon = %s ORDER BY waktu_pengiriman ASC 
                """,
                (no_wa, 0)
            )
            waktu_terakhir_kirim_permintaan = cursor.fetchone().get('MIN(waktu_pengiriman)')
            waktu_terakhir_kirim_permintaan
            # print()
            print(waktu_terakhir_kirim_permintaan)
            data_text = WA_API.check_new_respon(DRIVER,waktu_terakhir_kirim_permintaan)
            if data_text:
                print(data_text)
                valid_data = update_text_data(data_text,no_wa)
                if valid_data:
                    cursor.execute(
                        """
                        SELECT MAX(waktu_pengiriman) FROM log_pengiriman_permintaan 
                        WHERE no_wa = %s AND respon = %s ORDER BY waktu_pengiriman ASC 
                        """,
                        (no_wa, 0)
                    )
                    waktu_terakhir_kirim_permintaan_max = cursor.fetchone().get('MAX(waktu_pengiriman)')



                    cursor.execute(
                        """
                        UPDATE log_pengiriman_permintaan
                        SET respon = %s
                        WHERE no_wa = %s AND respon = %s AND waktu_pengiriman <= %s
                        """,
                        (1, no_wa, 0, waktu_terakhir_kirim_permintaan_max)
                    )
                    conn.commit()
                cursor.execute(
                    """
                    SELECT COUNT(*) AS jumlah_belum_respon FROM log_pengiriman_permintaan
                    WHERE no_wa = %s AND respon = %s
                    """,
                    (no_wa, 0)
                )
                result = cursor.fetchone()
                jumlah_belum_respon = result['jumlah_belum_respon'] if result else 0
                if jumlah_belum_respon <= 2:
                    jadwal_selanjutnya = time_now + JADWAL_SWITCH_OPERATOR.get('normal') 
                    pesan = f'''
                    Harap koresponden {no_wa} untuk segera mengirim data yang {jenis_data.get(pekerjaan)} untuk tanggal {time_now.strftime("%d/%m/%Y")}
                    '''
                else:
                    jadwal_selanjutnya = time_now + JADWAL_SWITCH_OPERATOR.get('eskalasi') 
                    pesan = f'''
                    Harap koresponden {no_wa} untuk segera mengirim data yang {jenis_data.get(pekerjaan)} periode tanggal {waktu_terakhir_kirim_permintaan} - {str(time_now)} 
                    ''' 
            
                kirim_pesan_permintaan(DRIVER,pesan)
                try:
                    cursor.execute(
                        "INSERT INTO log_pengiriman_permintaan (no_wa, waktu_pengiriman,respon) VALUES (%s, %s, %s)",
                        (no_wa, time_now, 0)
                    )
                    conn.commit()
                except Exception as e:
                    print(f"❌ Gagal insert respon: {e}")
                # perbarui jadwal
                try:
                    cursor.execute(
                        """
                        UPDATE jadwal_pengiriman_pesan_selanjutnya
                        SET jadwal_pengiriman_pesan_selanjutnya = %s
                        WHERE no_wa = %s
                        """,
                        (jadwal_selanjutnya, no_wa)
                    )
                    conn.commit()
                    print('✅ Jadwal pengiriman berhasil diperbarui')
                except Exception as e:
                    print(f"❌ Gagal update Jadwal: {e}")
                
            else:
                cursor.execute(
                    """
                    SELECT COUNT(*) AS jumlah_belum_respon FROM log_pengiriman_permintaan
                    WHERE no_wa = %s AND respon = %s
                    """,
                    (no_wa, 0)
                )
                result = cursor.fetchone()
                jumlah_belum_respon = result['jumlah_belum_respon'] if result else 0
                if jumlah_belum_respon <= 2:
                    jadwal_selanjutnya = time_now + JADWAL_SWITCH_OPERATOR.get('normal') 
                    pesan = f'''
                    Harap koresponden {no_wa} untuk segera mengirim data yang {jenis_data.get(pekerjaan)} untuk tanggal {time_now.strftime("%d/%m/%Y")}
                    '''
                else:
                    jadwal_selanjutnya = time_now + JADWAL_SWITCH_OPERATOR.get('eskalasi') 
                    pesan = f'''
                    Harap koresponden {no_wa} untuk segera mengirim data yang {jenis_data.get(pekerjaan)} periode tanggal {waktu_terakhir_kirim_permintaan} - {str(time_now)} 
                    ''' 
                
                kirim_pesan_permintaan(DRIVER,pesan)
                try:
                    print('insert into log')
                    cursor.execute(
                        "INSERT INTO log_pengiriman_permintaan (no_wa, waktu_pengiriman, respon) VALUES (%s, %s, %s)",
                        (no_wa, time_now, 0)
                    )
                    conn.commit()
                except Exception as e:
                    print(f"❌ Gagal insert respon: {e}")
                # perbarui jadwal
                try:
                    cursor.execute(
                        """
                        UPDATE jadwal_pengiriman_pesan_selanjutnya
                        SET jadwal_pengiriman_pesan_selanjutnya = %s
                        WHERE no_wa = %s
                        """,
                        (jadwal_selanjutnya, no_wa)
                    )
                    conn.commit()
                    print('✅ Jadwal pengiriman berhasil diperbarui')
                except Exception as e:
                    print(f"❌ Gagal update Jadwal: {e}")

        time.sleep(5)
except KeyboardInterrupt:
    print("\n🛑 Dihentikan oleh pengguna (Ctrl+C)")
        # update log pengiriman dengan nilai false dan terakhir_kirim < waktu_terakhir_kirim_permintaan menajdi True

    # df_pengiriman = pd.DataFrame(results, columns=['waktu_pengiriman'])
    # cursor.execute("SELECT MAX(waktu_pengiriman) AS terakhir_kirim FROM log_pengiriman_permintaan WHERE no_wa = %s AND", (no_wa,))
    # result = cursor.fetchone()
#     data_text = WA_API.check_new_respon(DRIVER,waktu_terakhir_kirim_permintaan)
#     data_text
#     if len(df_pengiriman['waktu_pengiriman'])>2:
#         if data_text:
#             update_text_data(data_text,formatted_wa)
    

#     else:
#         if data_text:
#             update_text_data(data_text,formatted_wa)
        



#     if len(results) <= 2:
#         data_text = WA_API.check_new_respon(DRIVER,waktu_terakhir_kirim_permintaan)
#         update_text_data(data_text,formatted_wa)
#         waktu_sekarang = datetime.now()
#         jadwal_baru = waktu_sekarang + timedelta(minutes=7)
#         cursor.execute(
#             "UPDATE jadwal_pengiriman_pesan_selanjutnya SET jadwal_pengiriman_pesan_selanjutnya = %s WHERE no_wa = %s",
#             (jadwal_baru, formatted_wa)
#         )
#         conn.commit()
#         if len(df['df_pengiriman'])>2:
#             pesan = f'koresponden {formatted_wa}, harap melengkapi data {jenis_data.get(pekerjaan)} periode {df['df_pengiriman'].min} - {waktu_sekarang}'
#         else:
#             pesan = f'koresponden {formatted_wa}, harap melengkapi data berikut {jenis_data.get(pekerjaan)}'
#         kirim_pesan_permintaan(DRIVER,pesan)
#     else:
#         data_text = WA_API.check_new_respon(DRIVER,waktu_terakhir_kirim_permintaan)
#         update_text_data(data_text,formatted_wa)
#         waktu_sekarang = datetime.now()
#         jadwal_baru = waktu_sekarang + timedelta(minutes=2)
#         cursor.execute(
#             "UPDATE jadwal_pengiriman_pesan_selanjutnya SET jadwal_pengiriman_pesan_selanjutnya = %s WHERE no_wa = %s",
#             (jadwal_baru, no_wa)
#         )
#         conn.commit()
#         pesan = f'koresponden {no_wa}, harap melengkapi data {jenis_data.get(pekerjaan)} periode {df['df_pengiriman'].min} - {waktu_sekarang}'
#         else:
#             pesan = f'koresponden {no_wa}, harap melengkapi data berikut {jenis_data.get(pekerjaan)}'
#         kirim_pesan_permintaan(DRIVER,pesan)
# else:
#     pass




Menjalankan proses... (Tekan Ctrl+C untuk berhenti)
2025-07-11 15:45:19
{datetime.datetime(2025, 7, 11, 12, 5): {'false_6285218274543@c.us_8A78B7130F2597083C01909757FED7F2': ['Siap bos']}, datetime.datetime(2025, 7, 11, 12, 10): {'false_6285218274543@c.us_3E72820D2C1E83B02861666FEF5FC8DE': ['Siap icik bos']}}
📩 Permintaan data terkirim.
✅ Jadwal pengiriman berhasil diperbarui
Menjalankan proses... (Tekan Ctrl+C untuk berhenti)
2025-07-11 15:45:19
{datetime.datetime(2025, 7, 11, 12, 5): {'false_6285218274543@c.us_8A78B7130F2597083C01909757FED7F2': ['Siap bos']}, datetime.datetime(2025, 7, 11, 12, 10): {'false_6285218274543@c.us_3E72820D2C1E83B02861666FEF5FC8DE': ['Siap icik bos']}}
📩 Permintaan data terkirim.
✅ Jadwal pengiriman berhasil diperbarui
Menjalankan proses... (Tekan Ctrl+C untuk berhenti)
2025-07-11 15:45:19
{datetime.datetime(2025, 7, 11, 12, 5): {'false_6285218274543@c.us_8A78B7130F2597083C01909757FED7F2': ['Siap bos']}, datetime.datetime(2025, 7, 11, 12, 10): {'false_62852